In [0]:
!pip install python_speech_features
!pip install python-dotenv 


  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5887 sha256=211f124af37a57ffc8dafb51a4217ab402129f6f76550dcca2836716b29745a3
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [0]:
!pip install keras --upgrade

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.3.1)


In [0]:
# !pip list librosa

In [0]:
import time
import numpy as np
import pandas as pd
from python_speech_features import mfcc
import os
from keras import layers
from keras import models
import keras
from keras import optimizers
from keras import regularizers
from keras import activations
from keras import backend as K
from keras.utils.data_utils import get_file

from sklearn.preprocessing import LabelEncoder, LabelBinarizer
import sys
sys.path.append('/content/drive/My Drive/cs577- Deep learning/deepMusic/')
import utils
from sklearn import preprocessing
import librosa
import matplotlib.pyplot as plt
import librosa.display
from sklearn.utils.class_weight import compute_class_weight
from math import floor


import tensorflow as tf
import keras.backend.tensorflow_backend as tfback
print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)

def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

tfback._get_available_gpus = _get_available_gpus



tf.__version__ is 2.2.0-rc3
tf.keras.__version__ is: 2.3.0-tf


In [0]:
# Small dataset
gen_params = {
          'dim': (96,1366),
          # 'dim': (96,469),
          'batch_size': 10,
          'n_classes': 4,
          'n_channels': 1,
          'shuffle': True}

root_folder = '/content/drive/My Drive/cs577- Deep learning/deepMusic/'
config = { 
    'audio_dir' : root_folder + 'fma_small',
    'tracks' : root_folder +'fma_metadata/subset_small.csv',
    'generator_params': gen_params,
    'audio_loader' : utils.FfmpegLoader()

}


In [0]:
def plot_mel_spect(spect):
    # spect = spect.flatten()
    # print(spect)
    plt.figure(figsize=(10, 5))
    librosa.display.specshow(spect.T, y_axis='mel', x_axis='time')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Test Melspectogram')
    plt.show()

In [0]:

# root_folder = "/content/drive/My Drive/cs577- Deep learning/deepMusic/"
# AUDIO_DIR = root_folder + 'fma_small'

# Read data
tracks = pd.read_csv(config['tracks'], index_col=0)
tracks = tracks[tracks.track_genre_top.isin(['Electronic', 'Folk', 'Rock','Hip-Hop'])]
# ['Electronic', 'Folk', 'Rock']
print(tracks['set_split'].shape)
train = tracks.loc[tracks['set_split'] == 'training']
val = tracks.loc[tracks['set_split'] == 'validation']
test = tracks.loc[tracks['set_split'] == 'test']
print(f'train -> {train.columns}')



(13126,)
train -> Index(['album_comments', 'album_date_created', 'album_date_released',
       'album_engineer', 'album_favorites', 'album_id', 'album_information',
       'album_listens', 'album_producer', 'album_tags', 'album_title',
       'album_tracks', 'album_type', 'artist_active_year_begin',
       'artist_active_year_end', 'artist_associated_labels', 'artist_bio',
       'artist_comments', 'artist_date_created', 'artist_favorites',
       'artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_members', 'artist_name', 'artist_related_projects',
       'artist_tags', 'artist_website', 'artist_wikipedia_page', 'set_split',
       'set_subset', 'track_bit_rate', 'track_comments', 'track_composer',
       'track_date_created', 'track_date_recorded', 'track_duration',
       'track_favorites', 'track_genre_top', 'track_genres',
       'track_genres_all', 'track_information', 'track_interest',
       'track_language_code', 'track_license', 'track_listen

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:

# Check which genres are present
genres = list(LabelEncoder().fit(train['track_genre_top']).classes_)
print('Top genres ({}): {}'.format(len(genres), genres))
le = LabelEncoder()
labels_encoded = np.asarray(le.fit_transform(tracks['track_genre_top']))
labels_encoded = pd.DataFrame(labels_encoded, index=tracks.index)

# print(f'label classes -> {le.classes_}')

y_integers = labels_encoded.to_numpy().flatten()
# print(f'labels_encoded -> {y_integers}')
class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
d_class_weights = dict(enumerate(class_weights))
# print(f'd_class_weights -> {d_class_weights}')

# For local training purposes
train = train.head(100)
val = val.head(100)
test = test.head(100)

# print(f'train y -> {train["track_genre_top"].unique()}')
# print(f'val y -> {val["track_genre_top"].unique()}')
# print(f'test y -> {test["track_genre_top"].unique()}')


Top genres (4): ['Electronic', 'Folk', 'Hip-Hop', 'Rock']


In [0]:
'''
IMPORTANT
'''

def compute_melgram(src, sr):
    ''' Compute a mel-spectrogram and returns it in a shape of (1,1,96,1366), where
    96 == #mel-bins and 1366 == #time frame

    parameters
    ----------
    audio_path: path for the audio file.
                Any format supported by audioread will work.
    More info: http://librosa.github.io/librosa/generated/librosa.core.load.html#librosa.core.load

    '''

    # mel-spectrogram parameters
    SR = 12000
    N_FFT = 512
    N_MELS = 96
    HOP_LEN = 256
    # DURA = 10
    DURA = 29.12  # to make it 1366 frame..

    # src, sr = librosa.load(audio_path, sr=SR)  # whole signal
    n_sample = src.shape[0]
    n_sample_fit = int(DURA*SR)
    # print(f'n_sample_fit -> {n_sample_fit}')
    # print(f'n_sample -> {n_sample}')
    # print(src[int((n_sample-n_sample_fit)/2):int((n_sample+n_sample_fit)/2)])


    if n_sample < n_sample_fit:  # if too short
        src = np.hstack((src, np.zeros((int(DURA*SR) - n_sample,))))
    elif n_sample > n_sample_fit:  # if too long
        src = src[int((n_sample-n_sample_fit)/2):int((n_sample+n_sample_fit)/2)]
    logam = librosa.amplitude_to_db
    melgram = librosa.feature.melspectrogram
    ret = logam(melgram(y=src, sr=SR, hop_length=HOP_LEN,
                        n_fft=N_FFT, n_mels=N_MELS)**2,
                ref=np.max)
    # plot_mel_spect(ret)
    ret = ret[np.newaxis, np.newaxis, :]
    return ret


In [0]:
'''
IMPORTANT
'''
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'

    def __init__(self, list_IDs, labels, loader, audio_dir, batch_size=32, dim=(32, 32, 32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.loader = loader
        self.audio_dir = audio_dir

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # print(list_IDs_temp)

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'  # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.n_channels, *self.dim))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # # Store sample
            # signal, rate = self.loader.load(utils.get_audio_path(self.audio_dir, ID))
            # sample = signal[:rate]
            
            # # Shorten the sample to 10 secs
            # ran_index = np.random.randint(0, signal.shape[0] - int(rate / 10))
            # sample = signal[ran_index:ran_index + int(rate / 10)]
            # # print(f'shape of sample -> {sample.shape}')
            # normalized_X = preprocessing.normalize(mfcc(sample, rate, numcep=13, nfilt=26, nfft=1103).T)
            # temp = np.array(normalized_X)
            # rshaped_X = temp.reshape(temp.shape[0], temp.shape[1], 1)
            # X[i,] = rshaped_X

            # Librosa version mel spectrogram
            signal, sr = librosa.load(utils.get_audio_path(self.audio_dir, ID))
            signal = signal.astype(float)
            spect = compute_melgram(signal, sr)
            X[i,] = spect
            
            # temp = np.array(spect)
            # rshaped_X = temp.reshape(temp.shape[0], temp.shape[1], 1)
            # X[i,] = rshaped_X
            # X[i,] = np.array(spect)

            # Store class
            y[i] = self.labels.loc[ID].to_numpy()

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)


In [0]:

# METHOD 2 - DATAGENERATOR
training_generator = DataGenerator(train.index.values, labels_encoded, config['audio_loader'], config['audio_dir'], **config['generator_params'])
x , y  = training_generator.__getitem__(0)
print(x.shape)
# print(train.loc[ID]['track_genre_top'])
print(y)
# Plot mel spect for audio file with ID 2
# plot_mel_spect(x[0],2)

In [0]:

def pop_layer(model):
    if not model.outputs:
        raise Exception('Sequential model cannot be popped: model is empty.')

    model.layers.pop()
    if not model.layers:
        model.outputs = []
        model.inbound_nodes = []
        model.outbound_nodes = []
    else:
        model.layers[-1].outbound_nodes = []
        model.outputs = [model.layers[-1].output]
    model.built = False


def MusicTaggerCRNN(weights='msd', input_tensor=None):
    '''Instantiate the MusicTaggerCRNN architecture,
    optionally loading weights pre-trained
    on Million Song Dataset. Note that when using TensorFlow,
    for best performance you should set
    `image_dim_ordering="tf"` in your Keras config
    at ~/.keras/keras.json.

    The model and the weights are compatible with both
    TensorFlow and Theano. The dimension ordering
    convention used by the model is the one
    specified in your Keras config file.

    For preparing mel-spectrogram input, see
    `audio_conv_utils.py` in [applications](https://github.com/fchollet/keras/tree/master/keras/applications).
    You will need to install [Librosa](http://librosa.github.io/librosa/)
    to use it.

    # Arguments
        weights: one of `None` (random initialization)
            or "msd" (pre-training on ImageNet).
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
    # Returns
        A Keras model instance.
    '''

    K.set_image_data_format('channels_first')
    if weights not in {'msd', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `msd` '
                         '(pre-training on Million Song Dataset).')

    # Determine proper input shape
    if K.image_data_format() == 'channels_first':
        input_shape = (1, 96, 1366)
    else:
        input_shape = (96, 1366, 1)

    if input_tensor is None:
        melgram_input = layers.Input(shape=input_shape)
    else:
        melgram_input = layers.Input(shape=input_tensor)

    # Determine input axis
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
        freq_axis = 2
        time_axis = 3
    else:
        channel_axis = 3
        freq_axis = 1
        time_axis = 2

    # Input block
    # x = layers.ZeroPadding2D(padding=(0, 37))(melgram_input)
    # x = layers.BatchNormalization(axis=time_axis, name='bn_0_freq')(x)

    # Conv block 1
    x = layers.Convolution2D(16,(2, 2), border_mode='same', name='conv1',activation= 'selu', trainable=True,kernel_initializer='lecun_normal')(melgram_input)
    x = layers.BatchNormalization(axis=channel_axis, mode=0, name='bn1', trainable=True)(x)
    # x = layers.ReLU()(x)
    x = layers.MaxPooling2D(pool_size=(4, 4), strides=(4, 4), name='pool1')(x)
    x = layers.AlphaDropout(0.1, name='dropout1', trainable=True)(x)

    # Conv block 2
    x = layers.Convolution2D(32,(2, 2), border_mode='same', name='conv2',activation= 'selu', trainable=True,kernel_initializer='lecun_normal')(x)
    x = layers.BatchNormalization(axis=channel_axis, mode=0, name='bn2', trainable=True)(x)
    # x = layers.ReLU()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool2')(x)
    x = layers.AlphaDropout(0.1, name='dropout2', trainable=True)(x)

    # # Conv block 3
    x = layers.Convolution2D(32,(3, 3), border_mode='same', name='conv3',activation= 'selu', trainable=True, kernel_initializer='lecun_normal')(x)
    x = layers.BatchNormalization(axis=channel_axis, mode=0, name='bn3', trainable=True)(x)
    # x = layers.ReLU()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool3')(x)
    x = layers.AlphaDropout(0.1, name='dropout3', trainable=True)(x)
    
    # model = models.Model(melgram_input, x)
    # model.summary()

    # # Conv block 4
    # x = layers.Convolution2D(16,( 4, 4), border_mode='same', name='conv4',activation= 'selu', trainable=True, kernel_initializer='lecun_normal')(x)
    # x = layers.BatchNormalization(axis=channel_axis, mode=0, name='bn4', trainable=True)(x)
    # # x = layers.ELU()(x)
    # x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool4')(x)
    # x = layers.Dropout(0.1, name='dropout4', trainable=True)(x)

    # reshaping
    if K.image_data_format() == 'channels_first':
        x = layers.Permute((3, 1, 2))(x)
    # print(f'permute size -> {x.shape}')
    shape = x.get_shape().as_list()
    x = layers.Reshape((shape[1]*shape[-1],shape[2]))(x)

    # GRU block 1, 2, output
    x = layers.GRU(8, return_sequences=True, name='gru1')(x)
    x = layers.GRU(8, return_sequences=False, name='gru2')(x)
    x = layers.AlphaDropout(0.3, name='final_drop')(x)

    ## LSTM Layer
    # layer = LSTM(96,return_sequences=False)(x)
    # x = Dropout(0.4)(layer)
    # print(f'lstm layer -> {layer.shape}')

    if weights is None:
        # x = layers.Flatten()(x)
        x = layers.Dense(128, activation='relu', name='hidden1')(x)
        x = layers.Dense(4, activation='softmax', name='output')(x)
        print(f'x.shape -> {x.shape}')

        model = models.Model(melgram_input, x)
        # model.summary()
        return model
        # return x
    else:
        # Load input
        x = layers.Dense(10, activation='sigmoid', name='output')(x)
        if K.image_data_format() == 'channels_last':
            raise RuntimeError("Please set image_dim_ordering == 'th'."
                               "You can set it at ~/.keras/keras.json")
        # Create model
        initial_model = models.Model(melgram_input, x)
        initial_model.load_weights('/content/crnn_net_gru_adam_ours_epoch_40.h5')

        # Eliminate last layer
        pop_layer(initial_model)
        pop_layer(initial_model)
        # Add new Dense layer
        last = initial_model.get_layer('gru2')
        preds = layers.Dense(4, activation='sigmoid', name='preds')(last.output)
        model = models.Model(initial_model.input, preds)

        # TO extend this model use this,
        # model = Model(initial_model.input, initial_model.get_layer('dropout2').output)

        return model

In [0]:
def plot_epocs_graph(history_dict,i):
    loss_vals = history_dict['loss']
    val_loss_vals = history_dict['val_loss']
    epochs = range(1, len(history_dict['acc']) + 1)
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss_vals, 'g', label='Training Loss')
    plt.plot(epochs, val_loss_vals, 'b', label='Validation Loss')
    plt.title("Training and validation loss")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    acc_vals = history_dict['acc']
    val_acc_vals = history_dict['val_acc']
    plt.plot(epochs, acc_vals, 'g', label='Training accuracy')
    plt.plot(epochs, val_acc_vals, 'b', label='Validation accuracy')
    plt.title("Training and validation Accuracy")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(root_folder + f'graphs/log{i}.png')
    plt.show()
    return

In [0]:
network_config = {
    'input_shape' : (1,96,1366),
    # 'input_shape' : (1,96,469),
    'loss' : 'categorical_crossentropy',
    'optimizer' : optimizers.Adam(learning_rate=0.001),
    'metrics' : ['acc'],
    'epochs' : 10,
    'batch_size':128,

}

In [0]:
# cnn
# model = gen_model()

# CRNN
# input_shape = (1,96,1366)
# model_input = layers.Input(input_shape, name='input')
# model = conv_recurrent_model_build(model_input,input_shape)

# CNN by mdeff
# input_shape = (1,96,704)
# model = getMdeff_model(input_shape)

# Another trial CRNN
model = MusicTaggerCRNN(weights=None, input_tensor=network_config['input_shape'])

model.compile(loss=network_config['loss'],
              optimizer=network_config['optimizer'],
              metrics=network_config['metrics'])
model.summary()




# Generators
training_generator = DataGenerator(train.index.values, labels_encoded,config['audio_loader'],config['audio_dir'], **config['generator_params'])
validation_generator = DataGenerator(val.index.values, labels_encoded,config['audio_loader'],config['audio_dir'], **config['generator_params'])
test_generator = DataGenerator(test.index.values, labels_encoded,config['audio_loader'],config['audio_dir'], **config['generator_params'])

# # Train model on dataset
train_val_history = model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    class_weight = d_class_weights,
                    epochs=network_config['epochs'],
                    use_multiprocessing=True,
                     workers=1)
history_dict = train_val_history.history
plot_epocs_graph(history_dict=history_dict)

print(f'Testing...')
test_op = model.evaluate_generator(generator = test_generator)
print('Testing accuracy -> ',test_op)



In [0]:

# def gen_model():
#   # Input shape is this because 13->numcep in mfcc function and 9 -> number of frames considered
#   # input_shape = (13, 9, 1)
#   input_shape = (64, 87, 1)

#   # Network Architecture
#   # tf.keras.backend.clear_session()

#   model = models.Sequential()

#   model.add(layers.Conv2D(16, (3, 3), activation='relu',
#                           strides=(1, 1), padding='same',
#                           input_shape=input_shape))
#   model.add(layers.Conv2D(32, (3, 3), activation='relu',
#                           strides=(1, 1), padding='same',
#                           ))
#   model.add(layers.MaxPool2D(2, 2))
#   model.add(layers.Dropout(0.5))
#   model.add(layers.Flatten())
#   model.add(layers.Dense(128, activation='relu'))
#   model.add(layers.Dense(64, activation='relu'))
#   model.add(layers.Dense(8, activation='softmax'))
#   model.summary()

#   # optimizer = optimizers.SGD(lr=0.1, momentum=0.9, nesterov=True)
#   optimizer = optimizers.Adam(lr=0.01)
#   model.compile(optimizer, loss='categorical_crossentropy', metrics=['acc'])
#   return model



In [0]:
'''
This is to be used when using all the classes

'''
# def pop_layer(model):
#     if not model.outputs:
#         raise Exception('Sequential model cannot be popped: model is empty.')

#     model.layers.pop()
#     if not model.layers:
#         model.outputs = []
#         model.inbound_nodes = []
#         model.outbound_nodes = []
#     else:
#         model.layers[-1].outbound_nodes = []
#         model.outputs = [model.layers[-1].output]
#     model.built = False


# def MusicTaggerCRNN(weights='msd', input_tensor=None):
#     '''Instantiate the MusicTaggerCRNN architecture,
#     optionally loading weights pre-trained
#     on Million Song Dataset. Note that when using TensorFlow,
#     for best performance you should set
#     `image_dim_ordering="tf"` in your Keras config
#     at ~/.keras/keras.json.

#     The model and the weights are compatible with both
#     TensorFlow and Theano. The dimension ordering
#     convention used by the model is the one
#     specified in your Keras config file.

#     For preparing mel-spectrogram input, see
#     `audio_conv_utils.py` in [applications](https://github.com/fchollet/keras/tree/master/keras/applications).
#     You will need to install [Librosa](http://librosa.github.io/librosa/)
#     to use it.

#     # Arguments
#         weights: one of `None` (random initialization)
#             or "msd" (pre-training on ImageNet).
#         input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
#             to use as image input for the model.
#     # Returns
#         A Keras model instance.
#     '''

#     K.set_image_data_format('channels_first')
#     if weights not in {'msd', None}:
#         raise ValueError('The `weights` argument should be either '
#                          '`None` (random initialization) or `msd` '
#                          '(pre-training on Million Song Dataset).')

#     # Determine proper input shape
#     if K.image_data_format() == 'channels_first':
#         input_shape = (1, 96, 1366)
#     else:
#         input_shape = (96, 1366, 1)

#     if input_tensor is None:
#         melgram_input = layers.Input(shape=input_shape)
#     else:
#         melgram_input = layers.Input(shape=input_tensor)

#     # Determine input axis
#     if K.image_data_format() == 'channels_first':
#         channel_axis = 1
#         freq_axis = 2
#         time_axis = 3
#     else:
#         channel_axis = 3
#         freq_axis = 1
#         time_axis = 2

#     # Input block
#     # x = layers.ZeroPadding2D(padding=(0, 37))(melgram_input)
#     # x = layers.BatchNormalization(axis=time_axis, name='bn_0_freq')(melgram_input)

#     # Conv block 1
#     x = layers.Convolution2D(64,(3, 3), border_mode='same', name='conv1', trainable=True)(melgram_input)
#     x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool1')(x)
#     x = layers.ELU()(x)
#     x = layers.Dropout(0.1, name='dropout1')(x)

#     # # Conv block 2
#     x = layers.Convolution2D(128,(3, 3), border_mode='same', name='conv2', trainable=True)(x)
#     x = layers.BatchNormalization(axis=channel_axis, mode=0, name='bn2')(x)
#     x = layers.ReLU()(x)
#     x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool2')(x)
#     x = layers.Dropout(0.1, name='dropout2')(x)

#     # # Conv block 3
#     x = layers.Convolution2D(128,( 3, 3),dilation_rate=2, border_mode='same', name='conv3', trainable=True)(x)
#     x = layers.BatchNormalization(axis=channel_axis, mode=0, name='bn3')(x)
#     x = layers.ReLU()(x)
#     x = layers.MaxPooling2D(pool_size=(3, 3), strides=(3, 3), name='pool3')(x)
#     x = layers.Dropout(0.1, name='dropout3')(x)
#     # model = models.Model(melgram_input, x)
#     # model.summary()
#     # # Conv block 4
#     x = layers.Convolution2D(128,( 3, 3), border_mode='same', name='conv4', trainable=True)(x)
#     x = layers.BatchNormalization(axis=channel_axis, mode=0, name='bn4')(x)
#     x = layers.ReLU()(x)
#     x = layers.MaxPooling2D(pool_size=(4, 4), strides=(4, 4), name='pool4')(x)
#     x = layers.Dropout(0.1, name='dropout4')(x)

#     # print(f'x size -> {x.shape}')
#     # reshaping
#     if K.image_data_format() == 'channels_first':
#         x = layers.Permute((3, 1, 2))(x)
#     # print(f'permute size -> {x.shape}')
#     shape = x.get_shape().as_list()
#     # print(f'shape  -> {(shape[0],shape[1]*shape[-1],shape[2])}')
#     # x = layers.Reshape((15, 128))(x)
#     x = layers.Reshape((shape[1]*shape[-1],shape[2]))(x)
#     # print(f'shape x -> {x.shape}')

#     # GRU block 1, 2, output
#     x = layers.GRU(64, return_sequences=True, name='gru1')(x)
#     x = layers.GRU(32, return_sequences=False, name='gru2')(x)
#     x = layers.Dropout(0.3, name='final_drop')(x)

#     ## LSTM Layer
#     # layer = LSTM(96,return_sequences=False)(x)
#     # x = Dropout(0.4)(layer)
#     # print(f'lstm layer -> {layer.shape}')

#     if weights is None:
#         # x = layers.Flatten()(x)
#         # x = Dense(512, activation='relu', name='hidden1')(x)
#         x = layers.Dense(4, activation='sigmoid', name='output')(x)
#         print(f'x.shape -> {x.shape}')

#         model = models.Model(melgram_input, x)
#         # model.summary()
#         return model
#         # return x
#     else:
#         # Load input
#         x = Dense(10, activation='sigmoid', name='output')(x)
#         if K.image_data_format() == 'channels_last':
#             raise RuntimeError("Please set image_dim_ordering == 'th'."
#                                "You can set it at ~/.keras/keras.json")
#         # Create model
#         initial_model = models.Model(melgram_input, x)
#         initial_model.load_weights('/content/crnn_net_gru_adam_ours_epoch_40.h5')

#         # Add new Dense layer
#         last = initial_model.get_layer('final_drop')
#         last = layers.Flatten()(last.output)
#         preds = layers.Dense(8, activation='softmax', name='preds')(last)
#         model = models.Model(initial_model.input, preds)

#         # TO extend this model use this,
#         # model = Model(initial_model.input, initial_model.get_layer('dropout2').output)

#         return model

In [0]:
# from keras import backend as K
# from keras.layers import Input, Dense
# from keras.models import Model
# from keras.layers import Dense, Dropout, Reshape, Permute, Flatten
# from keras.layers.convolutional import Convolution2D
# from keras.layers.convolutional import MaxPooling2D, ZeroPadding2D
# from keras.layers.normalization import BatchNormalization
# from keras.layers.advanced_activations import ELU
# from keras.layers.recurrent import GRU
# from keras.utils.data_utils import get_file

# def DeepMusicCRNN(input_layer=None):
#     '''
#     # Arguments
#         weights: one of `None` (random initialization)
#             or "msd" (pre-training on ImageNet).
#         input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
#             to use as image input for the model.
#     # Returns
#         A Keras model instance.
#     '''

#     K.set_image_data_format('channels_first')


#     # Determine input axis
#     if K.image_data_format() == 'channels_first':
#         channel_axis = 1
#         freq_axis = 2
#         time_axis = 3
#     else:
#         channel_axis = 3
#         freq_axis = 1
#         time_axis = 2

#     # input_x = Input(shape=input_tensor)
#     # Input block
#     # x = ZeroPadding2D(padding=(0, 37),input_shape = input_tensor)(input_x)
#     x = BatchNormalization(axis=time_axis, name='bn_0b_freq')(input_layer)

#     # # Conv block 1
#     x = Convolution2D(256, 3, 3, border_mode='same', name='conv1b', trainable=True)(x)
#     x = BatchNormalization(axis=channel_axis, mode=0, name='bn1b', trainable=True)(x)
#     x = ELU()(x)
#     x = MaxPooling2D(pool_size=(2, 2), strides=(3, 3), name='pool1b', trainable=True)(x)
#     x = Dropout(0.1, name='dropout1b', trainable=True)(x)

#     # # Conv block 2
#     x = Convolution2D(256, 3, 3, border_mode='same', name='conv2a', trainable=True)(x)
#     x = BatchNormalization(axis=channel_axis, mode=0, name='bn2a', trainable=True)(x)
#     x = ELU()(x)
#     x = MaxPooling2D(pool_size=(3, 3), strides=(3, 3), name='pool2a', trainable=True)(x)
#     x = Dropout(0.1, name='dropout2', trainable=True)(x)

#     # # Conv block 3
#     # x = Convolution2D(128, 3, 3, border_mode='same', name='conv3', trainable=False)(x)
#     # x = BatchNormalization(axis=channel_axis, mode=0, name='bn3', trainable=False)(x)
#     # x = ELU()(x)
#     # x = MaxPooling2D(pool_size=(4, 4), strides=(4, 4), name='pool3', trainable=False)(x)
#     # x = Dropout(0.1, name='dropout3', trainable=False)(x)

#     # # Conv block 4
#     # x = Convolution2D(128, 3, 3, border_mode='same', name='conv4', trainable=False)(x)
#     # x = BatchNormalization(axis=channel_axis, mode=0, name='bn4', trainable=False)(x)
#     # x = ELU()(x)
#     # x = MaxPooling2D(pool_size=(4, 4), strides=(4, 4), name='pool4', trainable=False)(x)
#     # x = Dropout(0.1, name='dropout4', trainable=False)(x)

#     # reshaping
#     # if K.image_data_format() == 'channels_first':
#     #     x = Permute((3, 1, 2))(x)
#     # x = Reshape((15, 256))(x)

#     # # # GRU block 1, 2, output
#     # x = GRU(32, return_sequences=True, name='gru1')(x)
#     # x = GRU(32, return_sequences=False, name='gru2')(x)
#     # x = Dropout(0.3, name='final_drop')(x)

#     # Create model
#     x = Flatten()(x)
#     x = Dense(16, activation='relu', name='hidden')(x)
#     x = Dense(8, activation='softmax', name='output')(x)
#     # model = Model(input_x, x)
#     # print(x.shape)
#     # # model.summary()
#     # return model
#     return x




In [0]:
# def getMdeff_model(input_shape):
#   model = models.Sequential()
#   # model.add(layers.Reshape((-1, 1), input_shape=input_shape))
#   # print(model.output_shape)

#   model.add(layers.Conv1D(128, 512, subsample_length=512,input_shape=input_shape))
#   # print(model.output_shape)
#   model.add(layers.Activation("relu"))

#   model.add(layers.Conv1D(32, 8))
#   # print(model.output_shape)
#   model.add(layers.Activation("relu"))
#   model.add(layers.MaxPooling1D(4))

#   model.add(layers.Conv1D(32, 8))
#   # print(model.output_shape)
#   model.add(layers.Activation("relu"))
#   model.add(layers.MaxPooling1D(4))

#   # print(model.output_shape)
#   #model.add(Dropout(0.25))
#   model.add(layers.Flatten())
#   print(model.output_shape)
#   model.add(layers.Dense(100))
#   model.add(layers.Activation("relu"))
#   # print(model.output_shape)
#   model.add(layers.Dense(labels_onehot.shape[1]))
#   model.add(layers.Activation("softmax"))
#   # print(model.output_shape)

#   optimizer = optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True)
#   #optimizer = keras.optimizers.Adam()#lr=1e-5)#, momentum=0.9, nesterov=True)
#   model.compile(optimizer, loss='categorical_crossentropy', metrics=['acc'])
#   return model

In [0]:
# num_classes = 8
# N_LAYERS = 3
# FILTER_LENGTH = 5
# CONV_FILTER_COUNT = 56
# BATCH_SIZE = 32
# LSTM_COUNT = 96
# EPOCH_COUNT = 70
# NUM_HIDDEN = 64
# L2_regularization = 0.001


# def conv_recurrent_model_build(model_input,input_shape):
#     print('Building model...')
#     layer = model_input
    
#     ### 3 1D Convolution Layers
#     for i in range(N_LAYERS):
#         # give name to the layers
#         layer = layers.Conv2D(
#                 filters=CONV_FILTER_COUNT,
#                 kernel_size=FILTER_LENGTH,
#                 kernel_regularizer=regularizers.l2(L2_regularization),  # Tried 0.001
#                 name='convolution_' + str(i + 1),
#                 data_format='channels_last',
#                 strides=1, padding='same',
#                 input_shape = input_shape
#             )(layer)
#         layer = layers.BatchNormalization(momentum=0.9)(layer)
#         layer = layers.Activation('relu')(layer)
#         layer = layers.MaxPooling2D(pool_size=2,padding='same',
#                 data_format='channels_last',)(layer)
#         layer = layers.Dropout(0.4)(layer)
    
#     # ## LSTM Layer
#     # layer = layers.LSTM(LSTM_COUNT, return_sequences=False)(layer)
#     # layer = layers.Dropout(0.4)(layer)
    
#     # ## Dense Layer
#     # layer = layers.Dense(NUM_HIDDEN, kernel_regularizer=regularizers.l2(L2_regularization), name='dense1')(layer)
#     # layer = layers.Dropout(0.4)(layer)
    
#     # ## Softmax Output
#     # layer = layers.Dense(num_classes)(layer)
#     # layer = layers.Activation('softmax', name='output_realtime')(layer)

#     model_output = layer
#     model = models.Model(model_input, model_output)
    
    
#     opt = optimizers.Adam(lr=0.001)
#     model.compile(
#             loss='categorical_crossentropy',
#             optimizer=opt,
#             metrics=['acc']
#         )
    
#     print(model.summary())
#     return model

In [0]:
# # Test the network
# print("Testing network on test set...")
# test_history = model.evaluate_generator(test_generator,use_multiprocessing=True, workers=1)

# print(test_history)